In [1]:
import pandas as pd
import numpy as np
import os

In [4]:
# chapter1 = pd.read_csv("C://Users//mynam//TariffBot//data//section1_data//htsdata_Chapter1.csv")

In [ ]:
chapter1.shape

(126, 9)

In [5]:
# chapter1.sample(10)

In [ ]:
# chapter2 = pd.read_csv("C:\\Users\\mynam\\TariffBot\\data\\section1_data\\htsdata_Chapter2.csv")

In [ ]:
# chapter3 = pd.read_csv("C:\\Users\\mynam\\TariffBot\\data\\section1_data\\htsdata_Chapter3.csv")
# chapter4 = pd.read_csv("C:\\Users\\mynam\\TariffBot\\data\\section1_data\\htsdata_Chapter4.csv")
# # chapter5 = pd.read_csv("C:\\Users\\mynam\\TariffBot\\data\\section1_data\\htsdata_Chapter5.csv")

In [6]:
# chapter2.head()

In [7]:
# chapter3.sample(6)

In [8]:
# chapter4.head()

In [9]:
# chapter5.head()

In [10]:
# merged_df = pd.concat([chapter1, chapter2, chapter3, chapter4, chapter5], ignore_index=True)

In [ ]:
# merged_df.columns

Index(['HTS Number', 'Indent', 'Description', 'Unit of Quantity',
       'General Rate of Duty', 'Special Rate of Duty', 'Column 2 Rate of Duty',
       'Quota Quantity', 'Additional Duties'],
      dtype='object')

In [ ]:
# merged_df.to_csv("section1.csv",index=False)

In [12]:
df = pd.read_csv("C:\\Users\\mynam\\TariffBot\\HTS-Tariff-Agent\\data\\downloads\\section1.csv")

In [13]:
df

,HTS Number,Indent,Description,Unit of Quantity,General Rate of Duty,Special Rate of Duty,Column 2 Rate of Duty,Quota Quantity,Additional Duties
0,0101,0,"Live horses, asses, mules and hinnies:",NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,1,Horses:,NaN,NaN,NaN,NaN,NaN,NaN
2,0101.21.00,2,Purebred breeding animals,NaN,Free,NaN,Free,NaN,NaN
3,0101.21.00.10,3,Males,"[""No.""]",NaN,NaN,NaN,NaN,NaN
4,0101.21.00.20,3,Females,"[""No.""]",NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
1664,0511.99.40.24,5,Dairy cattle,"[""No.""]",NaN,NaN,NaN,NaN,NaN
1665,0511.99.40.28,5,Other cattle,"[""No.""]",NaN,NaN,NaN,NaN,NaN
1666,0511.99.40.30,4,Dried blood,"[""kg""]",NaN,NaN,NaN,NaN,NaN
1667,0511.99.40.40,4,Other animal semen,"[""doses""]",NaN,NaN,NaN,NaN,NaN


In [14]:
import re

# Load HTS data CSV
# df = pd.read_csv("htsdata.csv")

# Input values
product_cost = 10000.0  # FOB cost
freight = 500.0
insurance = 100.0
unit_weight = 1000  # in kg
quantity = 10       # number of units

# Compute CIF
cif_value = product_cost + freight + insurance

# Duty parser
def parse_duty_advanced(duty_str, unit_weight=None, quantity=None):
    """
    Parses and returns a duty rate as a decimal fraction (e.g., 0.05 for 5%),
    or a calculated specific duty based on unit_weight or quantity.
    """
    if pd.isna(duty_str) or duty_str.strip() == "":
        return 0.0

    duty_str = duty_str.strip().lower()

    if "free" in duty_str:
        return 0.0

    # Percentage duty (e.g., '5%')
    match = re.search(r"([\d.]+)\s*%", duty_str)
    if match:
        return float(match.group(1)) / 100

    # Weight-based duty (e.g., '2.5¢/kg')
    match = re.search(r"([\d.]+)\s*¢/kg", duty_str)
    if match and unit_weight is not None:
        cents_per_kg = float(match.group(1))
        return (cents_per_kg * unit_weight) / (100 * cif_value)  # convert to % of CIF

    # Unit-based duty (e.g., '$1.00/unit')
    match = re.search(r"\$([\d.]+)/unit", duty_str)
    if match and quantity is not None:
        dollars_per_unit = float(match.group(1))
        return (dollars_per_unit * quantity) / cif_value

    return 0.0

# Create working DataFrame
duty_df = df[["HTS Number", "Description", "General Rate of Duty", 
              "Special Rate of Duty", "Column 2 Rate of Duty"]].copy()

# Apply CIF and input data
duty_df["CIF Value"] = cif_value
duty_df["Product Cost"] = product_cost
duty_df["Freight"] = freight
duty_df["Insurance"] = insurance

# Apply duty calculations
for col in ["General Rate of Duty", "Special Rate of Duty", "Column 2 Rate of Duty"]:
    parsed_col = f"{col} Parsed (%)"
    amount_col = f"{col} Duty Amount"
    duty_df[parsed_col] = duty_df[col].apply(lambda x: parse_duty_advanced(x, unit_weight, quantity))
    duty_df[amount_col] = duty_df[parsed_col] * cif_value

# Filter results with any applicable duty
duty_df_filtered = duty_df[
    (duty_df["General Rate of Duty Duty Amount"] > 0) |
    (duty_df["Special Rate of Duty Duty Amount"] > 0) |
    (duty_df["Column 2 Rate of Duty Duty Amount"] > 0)
]

# Save results
duty_df_filtered.to_csv("landed_cost_duties.csv", index=False)


In [15]:
landed_cost = pd.read_csv("landed_cost_duties.csv")

In [18]:
landed_cost.Description

0                                                  Other
1                                                  Asses
2                                                  Other
3             Cows imported specially for dairy purposes
4                                                  Other
                             ...                        
513                 Ambergris, castoreum, civet and musk
514                                         Bovine semen
515    Products chiefly used as food for animals or a...
516                     Natural sponges of animal origin
517                                                Other
Name: Description, Length: 518, dtype: object